In [307]:
#Importing the libraries

from IPython.display import display, HTML
import csv
import re
import pandas as pd
import matplotlib.pyplot as plt
import sqlite3
from sqlite3 import Error

In [308]:
### Utility Functions

def create_connection(db_file, delete_db=False):
    import os
    if delete_db and os.path.exists(db_file):
        os.remove(db_file)

    conn = None
    try:
        conn = sqlite3.connect(db_file)
        conn.execute("PRAGMA foreign_keys = 1")
    except Error as e:
        print(e)

    return conn


def create_table(conn, create_table_sql, drop_table_name=None):
    
    if drop_table_name: 
        try:
            c = conn.cursor()
            c.execute("""DROP TABLE IF EXISTS %s""" % (drop_table_name))
        except Error as e:
            print(e)
    
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except Error as e:
        print(e)
        
def execute_sql_statement(sql_statement, conn):
    cur = conn.cursor()
    cur.execute(sql_statement)

    rows = cur.fetchall()

    return rows

In [309]:
# Functions To Insert Into Tables

def insert_project_details(conn,values):
    sql = """ INSERT INTO ProjectDetails VALUES(?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?,?) """
    cur = conn.cursor()
    cur.executemany(sql,values)
    return cur.lastrowid

def insert_project(conn,values):
    sql = """ INSERT INTO Project VALUES(?,?,?,?,?,?,?,?) """
    cur = conn.cursor()
    cur.executemany(sql,values)
    return cur.lastrowid

def insert_location(conn,values):
    sql = """ INSERT INTO ProjectLocation VALUES(NULL,?,?,?,?) """
    cur = conn.cursor()
    cur.executemany(sql,values)
    return cur.lastrowid

def insert_inverter(conn,values):
    sql = """ INSERT INTO InverterDetails VALUES(NULL,?,?,?,?) """
    cur = conn.cursor()
    cur.executemany(sql,values)
    return cur.lastrowid

def insert_pvmodule(conn,values):
    sql = """ INSERT INTO PVModuleDetails VALUES(NULL,?,?,?,?) """
    cur = conn.cursor()
    cur.executemany(sql,values)
    return cur.lastrowid

def insert_project_timeline(conn,values):
    sql = """ INSERT INTO ProjectTimeline VALUES(NULL,?,?,?) """
    cur = conn.cursor()
    cur.executemany(sql,values)
    return cur.lastrowid

def insert_project_cost_info(conn,values):
    sql = """ INSERT INTO ProjectCost VALUES(NULL,?,?,?,?,?,?) """
    cur = conn.cursor()
    cur.executemany(sql,values)
    return cur.lastrowid

def insert_project_production_info(conn,values):
    sql = """ INSERT INTO ProjectProd VALUES(NULL,?,?,?,?) """
    cur = conn.cursor()
    cur.executemany(sql,values)
    return cur.lastrowid

In [310]:
# Delete old database file
normalized_database_filename = 'normalized.db'
conn = create_connection(normalized_database_filename, delete_db=True)

In [311]:
# Cleansing City Data
def cleanse(city):
    digit_checker = any(chr.isdigit() for chr in city)
    
    city = city.lower()
    
    if city.endswith(','):
        city = city.replace(',','')
    
    if city.endswith(' '):
        city = city.replace(' ','')
        
    if ',' in city:
        city_modified = re.search("^([a-z]*).*$",city)
        city_modified = str(city_modified.group(1))
        city = city.replace(city,city_modified)
    
    if digit_checker == True:
        city = ''
        
    if city == "Bardinia":
        city = city.replace('Bardinia','Bardonia')
        
    if city == 'cabria hgts':
        city = city.replace('cabria hgts','cambria heights')
        
    if city == 'jackson hgts ny':
        city = city.replace('jackson hgts ny','jackson heights')
        
    if ' hts' in city:
        city = city.replace(' hts',' heights')
        
    if ' hgts' in city:
        city = city.replace(' hgts',' heights')
    
    return city.title()

In [312]:
# Creating a dictionary to load the data into the table columns.
def asSolarInfo(row):
    
    loc_coor = row[30]
    loc_coor_modified = re.search("[^(]*([+-]?\d+\.\d+, *[+-]?\d+.\d+)", loc_coor)
    loc_coor_modified = "(" + loc_coor_modified.group(0) + ")"
    city = cleanse(row[3])
    solar_details_dict = {'reporting_period':row[0],'project_id':row[1],'city':city,
                     'county':row[4],'state':row[5],'zip_code':row[6],'sector':row[7],'program_type':row[8],
                     'solicitation':row[9],'electric_utility':row[10],'purchase_type':row[11],
                     'date_application_recieved':row[12],'date_completed':row[13],
                     'project_status':row[14],'contractor':row[15],
                     'primary_inverter_manufacturer':row[16],
                     'primary_inverter_model_number':row[17],'total_inverter_quantity':row[18],
                     'primary_pv_module_manufacturer':row[19],
                     'pv_module_model_number':row[20],'total_pv_module_quantity':row[21],
                     'project_cost':row[22],'incentive':row[23],'total_nameplate_kw_dc':row[24],
                     'expected_kwh_annual_production':row[25],'remote_net_metering':row[26],
                     'affordable_solar':row[27],'community_distributed_generation':row[28],
                     'green_jobs_green_ny_participants':row[29],'loc_coordinates':loc_coor_modified}
    return solar_details_dict

In [313]:
# CREATING THE NON-NORMALIZED DATABASE

def non_normalized_db(data_filename, normalized_database_filename):
    first = True
    reporting_period_list = []
    project_id_list = []
    city_list = []
    county_list = []
    state = []
    zip_code_list = []
    sector_list = []
    program_type_list = []
    solicitation_list = []
    electric_utility_list = []
    purchase_type_list = []
    date_application_recieved_list = []
    date_completed_list = []
    project_status_list = []
    contractor_list = []
    inverter_manufacturer_list = []
    inverter_model_no_list = []
    total_inverter_quantity_list = []
    pv_module_manufacturer_list = []
    pv_module_model_number_list = []
    total_pv_module_quantity_list = []
    project_cost_list = []
    incentive_list = []
    total_nameplate_kw_dc_list = []
    expected_kwh_annual_production_list = []
    remote_net_metering_list = []
    affordable_solar_list = []
    community_distributed_generation_list = []
    green_jobs_green_ny_participants_list = []
    loc_coordinates_list = []
    with open(data_filename) as file:
        file_reader = csv.reader(file, delimiter=',')
        for row in file_reader:
            if first:
                    first = False
                    continue
            info = asSolarInfo(row)
            reporting_period_list.append(info['reporting_period'])
            project_id_list.append(info['project_id'])
            city_list.append(info['city'])
            county_list.append(info['county'])
            state.append(info['state'])
            zip_code_list.append(info['zip_code'])
            sector_list.append(info['sector'])
            program_type_list.append(info['program_type'])
            solicitation_list.append(info['solicitation'])
            electric_utility_list.append(info['electric_utility'])
            purchase_type_list.append(info['purchase_type'])
            date_application_recieved_list.append(info['date_application_recieved'])
            date_completed_list.append(info['date_completed'])
            project_status_list.append(info['project_status'])
            contractor_list.append(info['contractor'])
            inverter_manufacturer_list.append(info['primary_inverter_manufacturer'])
            inverter_model_no_list.append(info['primary_inverter_model_number'])
            total_inverter_quantity_list.append(info['total_inverter_quantity'])
            pv_module_manufacturer_list.append(info['primary_pv_module_manufacturer'])
            pv_module_model_number_list.append(info['pv_module_model_number'])
            total_pv_module_quantity_list.append(info['total_pv_module_quantity'])
            project_cost_list.append(info['project_cost'])
            incentive_list.append(info['incentive'])
            total_nameplate_kw_dc_list.append(info['total_nameplate_kw_dc'])
            expected_kwh_annual_production_list.append(info['expected_kwh_annual_production'])
            remote_net_metering_list.append(info['remote_net_metering'])
            affordable_solar_list.append(info['affordable_solar'])
            community_distributed_generation_list.append(info['community_distributed_generation'])
            green_jobs_green_ny_participants_list.append(info['green_jobs_green_ny_participants'])
            loc_coordinates_list.append(info['loc_coordinates'])
        
    solar_project_details = zip(reporting_period_list,project_id_list,city_list,county_list,
                                state,zip_code_list,sector_list,program_type_list,
                                solicitation_list,electric_utility_list,purchase_type_list,
                                date_application_recieved_list,date_completed_list,
                                project_status_list,contractor_list,
                                inverter_manufacturer_list,inverter_model_no_list,
                                total_inverter_quantity_list,pv_module_manufacturer_list,
                                pv_module_model_number_list,total_pv_module_quantity_list,
                                project_cost_list,incentive_list,total_nameplate_kw_dc_list,
                                expected_kwh_annual_production_list,remote_net_metering_list,
                                affordable_solar_list,community_distributed_generation_list,
                                green_jobs_green_ny_participants_list,loc_coordinates_list)
        
    project_details_table = """CREATE TABLE ProjectDetails
    ([ReportingPeriod] TEXT,
    [ProjectID] TEXT PRIMARY KEY, 
    [City] TEXT,
    [County] TEXT,
    [State] TEXT,
    [ZipCode] TEXT,
    [Sector] TEXT,
    [ProgramType] TEXT,
    [Solicitation] TEXT,
    [ElectricUtility] TEXT,
    [PurchaseType] TEXT,
    [DateRecieved] TEXT,
    [DateCompleted] TEXT,
    [ProjectStatus] TEXT,
    [Contractor] TEXT,
    [InverterManufacturer] TEXT,
    [InverterModelNumber] TEXT,
    [TotalInverterQuantity] TEXT,
    [PvModuleManufacturer] TEXT,
    [PvModuleModelNumber] TEXT,
    [TotalPvModuleQuantity] TEXT,
    [ProjectCost] TEXT,
    [Incentive] TEXT,
    [TotalNameplateKwDc] TEXT,
    [ExpectedKwhAnnualProduction] TEXT,
    [RemoteNetMetering] TEXT,
    [AffordableSolar] TEXT,
    [CommunityDistrbutedGeneration] TEXT,
    [GreenJobsParticipants] TEXT,
    [LocCoordinates] TEXT
    );"""
    create_table(conn, project_details_table,True)
    insert_project_details(conn,solar_project_details)
    conn.commit()

In [314]:
#Verifying the table creation & insertion of Non-Normalized DB

data_filename = '/Users/raj/Desktop/Project/solar.csv'
normalized_database_filename = 'normalized.db'
non_normalized_db(data_filename, normalized_database_filename)
conn = create_connection(normalized_database_filename)
cur = conn.cursor()
df = pd.read_sql_query("""SELECT * FROM ProjectDetails""", conn)

In [315]:
#Creating the Project Table

def create_project_table(data_filename,normalized_database_filename):
    first = True
    project_id_list = []
    sector_list = []
    program_type_list = []
    solicitation_list = []
    electric_utility_list = []
    purchase_type_list = []
    project_status_list = []
    contractor_list = []
    with open(data_filename) as file:
        file_reader = csv.reader(file, delimiter=',')
        for row in file_reader:
            if first:
                    first = False
                    continue
            info = asSolarInfo(row)
            project_id_list.append(info['project_id'])
            sector_list.append(info['sector'])
            program_type_list.append(info['program_type'])
            solicitation_list.append(info['solicitation'])
            electric_utility_list.append(info['electric_utility'])
            purchase_type_list.append(info['program_type'])
            project_status_list.append(info['project_status'])
            contractor_list.append(info['contractor'])
            
    project_table = """CREATE TABLE Project
    ([ProjectID] TEXT NOT NULL PRIMARY KEY,
    [Sector] TEXT,
    [Program Type] TEXT,
    [Solicitation] TEXT,
    [ElectricUtility] TEXT,
    [PurchaseType] TEXT,
    [ProjectStatus] TEXT,
    [Contractor] TEXT,
    UNIQUE(ProjectID));"""
    
    project_details = zip(project_id_list,sector_list,program_type_list,solicitation_list,
                          electric_utility_list,purchase_type_list,project_status_list,contractor_list)
    
    create_table(conn, project_table,True)
    insert_project(conn,project_details)
    conn.commit()

In [316]:
#Verifying the table creation & insertion of Project

data_filename = '/Users/raj/Desktop/Project/solar.csv'
normalized_database_filename = 'normalized.db'
create_project_table(data_filename, normalized_database_filename)
conn = create_connection(normalized_database_filename)
cur = conn.cursor()
df = pd.read_sql_query("""SELECT * FROM Project""", conn)

In [317]:
#Creating the ProjectLocation Table

def create_project_location_table(data_filename, normalized_database_filename):
    first = True
    project_id_list = []
    city_list = []
    county_list = []
    zip_code_list = []
    with open(data_filename) as file:
        file_reader = csv.reader(file, delimiter=',')
        for row in file_reader:
            if first:
                    first = False
                    continue
            info = asSolarInfo(row)
            city = cleanse(info['city'])
            project_id_list.append(info['project_id'])
            city_list.append(city)
            county_list.append(info['county'])
            zip_code_list.append(info['zip_code'])

    location_details = zip(zip_code_list,project_id_list,city_list,county_list)
    
                
    project_location_table = """create table ProjectLocation
    ([ProjectLocationID] INTEGER NOT NULL PRIMARY KEY,
    [ZipCode] TEXT NOT NULL,
    [ProjectID] TEXT NOT NULL,
    [City] TEXT,
    [County] TEXT,
    FOREIGN KEY(ProjectID) REFERENCES Project(ProjectID),
    UNIQUE(ProjectLocationID)
    );"""
    
    create_table(conn, project_location_table,True)
    insert_location(conn,location_details)
    conn.commit()
                

In [318]:
#Verifying the table creation & insertion of ProjectLocation

data_filename = '/Users/raj/Desktop/Project/solar.csv'
normalized_database_filename = 'normalized.db'
create_project_location_table(data_filename, normalized_database_filename)
conn = create_connection(normalized_database_filename)
cur = conn.cursor()
df = pd.read_sql_query("""SELECT * FROM ProjectLocation order by City""", conn)

In [319]:
#Creating the InverterDetails Table

def create_inverter_details_table(data_filename, normalized_database_filename):
    first = True
    project_id_list = []
    PriInvetMft_list = []
    PriInvetModNo_list = []
    TotalInvtQt_list = []
    
    with open(data_filename) as file:
        file_reader = csv.reader(file, delimiter=',')
        for row in file_reader:
            if first:
                    first = False
                    continue
            info = asSolarInfo(row)
            project_id_list.append(info['project_id'])
            PriInvetMft_list.append(info['primary_inverter_manufacturer'])
            PriInvetModNo_list.append(info['primary_inverter_model_number'])
            TotalInvtQt_list.append(info['total_inverter_quantity'])
            
    inverter_details = zip(project_id_list,PriInvetMft_list,PriInvetModNo_list,TotalInvtQt_list)
    
    inverter_details_table = """CREATE TABLE InverterDetails
    ([InverterDetailsID] INTEGER NOT NULL PRIMARY KEY, 
    [ProjectID] TEXT NOT NULL,
    [PriInvetMft] TEXT, 
    [PriInvetModNo] TEXT,
    [TotalInvtQt] TEXT,
    FOREIGN KEY(ProjectID) REFERENCES Project(ProjectID),
    UNIQUE(InverterDetailsID));"""
    
    create_table(conn, inverter_details_table,True)
    insert_inverter(conn,inverter_details)
    conn.commit()
    

In [320]:
#Verifying the table creation & insertion of InverterDetails

data_filename = '/Users/raj/Desktop/Project/solar.csv'
normalized_database_filename = 'normalized.db'
create_inverter_details_table(data_filename, normalized_database_filename)
conn = create_connection(normalized_database_filename)
cur = conn.cursor()
df = pd.read_sql_query("""SELECT * FROM  InverterDetails""", conn)

In [321]:
#Creating the PVModuleDetails Table

def create_pvmodule_details_table(data_filename, normalized_database_filename):
    first = True
    project_id_list = []
    PriPVMft_list = []
    PriPVModNo_list = []
    TotalPVQt_list = []
    
    with open(data_filename) as file:
        file_reader = csv.reader(file, delimiter=',')
        for row in file_reader:
            if first:
                    first = False
                    continue
            info = asSolarInfo(row)
            project_id_list.append(info['project_id'])
            PriPVMft_list.append(info['primary_pv_module_manufacturer'])
            PriPVModNo_list.append(info['pv_module_model_number'])
            TotalPVQt_list.append(info['total_pv_module_quantity'])
            
    pvmodule_details = zip(project_id_list,PriPVMft_list,PriPVModNo_list,TotalPVQt_list)
    
    pvmodule_table = """CREATE TABLE PVModuleDetails
    ([PVModuleDetailsID] INTEGER NOT NULL PRIMARY KEY, 
    [ProjectID] TEXT NOT NULL,
    [PriPVMft] TEXT,
    [PriPVModNo] TEXT,
    [TotalPVQt] TEXT, 
    FOREIGN KEY(ProjectID) REFERENCES Project(ProjectID),
    UNIQUE(PVModuleDetailsID));"""
    
    create_table(conn,pvmodule_table,True)
    insert_pvmodule(conn,pvmodule_details)
    conn.commit()

In [322]:
#Verifying the table creation & insertion of PVModuleDetails

data_filename = '/Users/raj/Desktop/Project/solar.csv'
normalized_database_filename = 'normalized.db'
create_pvmodule_details_table(data_filename, normalized_database_filename)
conn = create_connection(normalized_database_filename)
cur = conn.cursor()
df = pd.read_sql_query("""SELECT * FROM  PVModuleDetails""", conn)

In [323]:
#Creating the ProjectTimeline Table

def create_project_timeline_table(data_filename, normalized_database_filename):
    first = True
    project_id_list = []
    DtAppReceived_list = []
    DtCompleted_list = []
    
    with open(data_filename) as file:
        file_reader = csv.reader(file, delimiter=',')
        for row in file_reader:
            if first:
                    first = False
                    continue
            info = asSolarInfo(row)
            project_id_list.append(info['project_id'])
            DtAppReceived_list.append(info['date_application_recieved'])
            DtCompleted_list.append(info['date_completed'])
            
    project_timeline_details = zip(project_id_list,DtAppReceived_list,DtCompleted_list)
    
    project_timeline_table = """CREATE TABLE ProjectTimeline
    ([ProjectTimelineID] INTEGER NOT NULL PRIMARY KEY,
    [ProjectID] TEXT NOT NULL,
    [DtAppReceived] TEXT,
    [DtCompleted] TEXT,
    FOREIGN KEY(ProjectID) REFERENCES Project(ProjectID),
    UNIQUE(ProjectTimelineID));"""
    
    create_table(conn,project_timeline_table,True)
    insert_project_timeline(conn,project_timeline_details)
    conn.commit()

In [324]:
#Verifying the table creation & insertion of ProjectTimeline

data_filename = '/Users/raj/Desktop/Project/solar.csv'
normalized_database_filename = 'normalized.db'
create_project_timeline_table(data_filename, normalized_database_filename)
conn = create_connection(normalized_database_filename)
cur = conn.cursor()
df = pd.read_sql_query("""SELECT * FROM  ProjectTimeline""", conn)

In [325]:
#Creating the ProjectCost Table

def create_project_cost_table(data_filename, normalized_database_filename):
    first = True
    project_id_list = []
    project_cost_list = []
    incentive_list = []
    RemNetMet_list = []
    AffSolar_list = []
    GreenCertified_list = []
    
    with open(data_filename) as file:
        file_reader = csv.reader(file, delimiter=',')
        for row in file_reader:
            if first:
                    first = False
                    continue
            info = asSolarInfo(row)
            project_id_list.append(info['project_id'])
            project_cost_list.append(info['project_cost'])
            incentive_list.append(info['incentive'])
            RemNetMet_list.append(info['remote_net_metering'])
            AffSolar_list.append(info['affordable_solar'])
            GreenCertified_list.append(info['green_jobs_green_ny_participants'])
            
    project_cost_details = zip(project_id_list,project_cost_list,incentive_list,RemNetMet_list,
                               AffSolar_list,GreenCertified_list)
    
    project_cost_table = """CREATE TABLE ProjectCost
    ([ProjectCostID] INTEGER NOT NULL PRIMARY KEY,
    [ProjectID] TEXT NOT NULL,
    [ProjectCost] TEXT,
    [Incentive] TEXT,
    [RemNetMet] TEXT,
    [AffSolar] TEXT,
    [GreenCertified] TEXT,
    FOREIGN KEY(ProjectID) REFERENCES Project(ProjectID),
    UNIQUE(ProjectCostID));"""
    
    create_table(conn,project_cost_table,True)
    insert_project_cost_info(conn,project_cost_details)
    conn.commit()

In [326]:
#Verifying the table creation & insertion of ProjectCost

data_filename = '/Users/raj/Desktop/Project/solar.csv'
normalized_database_filename = 'normalized.db'
create_project_cost_table(data_filename, normalized_database_filename)
conn = create_connection(normalized_database_filename)
cur = conn.cursor()
df = pd.read_sql_query("""SELECT * FROM  ProjectCost""", conn)

In [327]:
#Creating the ProjectProd Table

def create_project_prod_table(data_filename, normalized_database_filename):
    first = True
    project_id_list = []
    TotNamePlateKWDC_list = []
    ExpectKWhAnnProd_list = []
    CommDistGener_list = []
    
    with open(data_filename) as file:
        file_reader = csv.reader(file, delimiter=',')
        for row in file_reader:
            if first:
                    first = False
                    continue
            info = asSolarInfo(row)
            project_id_list.append(info['project_id'])
            TotNamePlateKWDC_list.append(info['total_nameplate_kw_dc'])
            ExpectKWhAnnProd_list.append(info['expected_kwh_annual_production'])
            CommDistGener_list.append(info['community_distributed_generation'])
            
    project_producation_details = zip(project_id_list,TotNamePlateKWDC_list,ExpectKWhAnnProd_list,
                                      CommDistGener_list)
    
    project_production_table = """CREATE TABLE ProjectProd
    ([ProjectProdID] INTEGER NOT NULL PRIMARY KEY,
    [ProjectID] TEXT NOT NULL,
    [TotNamePlateKWDC] TEXT, 
    [ExpectKWhAnnProd] TEXT,
    [CommDistGener] TEXT,
    FOREIGN KEY(ProjectID) REFERENCES Project(ProjectID),
    UNIQUE(ProjectProdID));"""
    
    create_table(conn,project_production_table,True)
    insert_project_production_info(conn,project_producation_details)
    conn.commit()

In [328]:
#Verifying the table creation & insertion of ProjectProd

data_filename = '/Users/raj/Desktop/Project/solar.csv'
normalized_database_filename = 'normalized.db'
create_project_prod_table(data_filename, normalized_database_filename)
conn = create_connection(normalized_database_filename)
cur = conn.cursor()
df = pd.read_sql_query("""SELECT * FROM  ProjectProd""", conn)